This is following:  A Neural Algorithm of Artistic Style

Leon A. Gatys, Alexander S. Ecker, Matthias Bethge

--> https://arxiv.org/abs/1508.06576

Style transfer is an optimisation probelem, to  minimisation the loss function --> taking two images (style image and whatever x image ), then to minimise the difference betweeen the images. 


Zero shot or one shot learning :: machine learning with very little data. 

Network used is VGG16 : it is an image classification convoltionary network 
At each layer it would apply a series of operations to the input image
Input image: matrix of values 
At each layer we have a stack of filters that are learned overtime

Filters: the filters are 3D vectors. Thye are a collection of matrics that are two-d and the 3D is the rgb
At each of these filters, the filter is multiplied by the input image. SO it is a matrox multiplication and then a summation operation.
If it dects a fecature = very latge number
If not = zero , therefore it is a feature identifier. 

It will output a feature map or an activation map ( huge matrix of values) . We want to minimise te difefre between our feature map and our content is


Conctent image : want to modify 
Style image: we want to apply to conetcnt imag e
Mixed Image: empty, radom nosie initilase dimage we will add to over time.

There is style oss and contnet loss and combination them usign gradient value ot updat emixed image.


In [ ]:
# import our dependencies 

%matplotlib inline
import matplotlib.pyplot as plt # plot out what we see
import tensorflow as tf  # machine learnign library 
import numpy as np   # help calculate gram matrix
import PIL.Image    # show image we want to show

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
#16 layer convostuionary netwoerk. It is prerained with neccesssary filters
# more layers means better predictions but mroe computation time 
import vgg16.py

ModuleNotFoundError: ignored

In [ ]:
from keras.applications.vgg16 import VGG16
model = VGG16(weights='imagenet')
print(model.summary())

553476096/553467096 [==============================] - 8s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [ ]:
vgg16.maybe_download()

NameError: ignored

In [ ]:
# define image helper fucntions 

def load_image(filename, max_size=None):
    image = PIL.Image.open(filename)

    if max_size is not None:
        # Calculate the appropriate rescale-factor for
        # ensuring a max height and width, while keeping
        # the proportion between them.
        factor = max_size / np.max(image.size)
    
        # Scale the image's height and width.
        size = np.array(image.size) * factor

        # The size is now floating-point because it was scaled.
        # But PIL requires the size to be integers.
        size = size.astype(int)

        # Resize the image.
        image = image.resize(size, PIL.Image.LANCZOS)

    # Convert to numpy floating-point array.
    return np.float32(image)

In [ ]:
def save_image(image, filename):
    # Ensure the pixel-values are between 0 and 255.
    image = np.clip(image, 0.0, 255.0)
    
    # Convert to bytes.
    image = image.astype(np.uint8)
    
    # Write the image-file in jpeg-format.
    with open(filename, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')

In [ ]:
def plot_image_big(image):
    # Ensure the pixel-values are between 0 and 255.
    image = np.clip(image, 0.0, 255.0)

    # Convert pixels to bytes.
    image = image.astype(np.uint8)

    # Convert to a PIL-image and display it.
    display(PIL.Image.fromarray(image))


In [ ]:
def plot_images(content_image, style_image, mixed_image):
    # Create figure with sub-plots.
    fig, axes = plt.subplots(1, 3, figsize=(10, 10))

    # Adjust vertical spacing.
    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    # Use interpolation to smooth pixels?
    smooth = True
    
    # Interpolation type.
    if smooth:
        interpolation = 'sinc'
    else:
        interpolation = 'nearest'

    # Plot the content-image.
    # Note that the pixel-values are normalized to
    # the [0.0, 1.0] range by dividing with 255.
    ax = axes.flat[0]
    ax.imshow(content_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Content")

    # Plot the mixed-image.
    ax = axes.flat[1]
    ax.imshow(mixed_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Mixed")

    # Plot the style-image
    ax = axes.flat[2]
    ax.imshow(style_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Style")

    # Remove ticks from all the plots.
    for ax in axes.flat:
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

Loss Function 



In [ ]:
# define mean squared error:  it is a function given to input tensors (a, b)
# outputs a scaler --> single value

# it is a hlerper fucntion inside our content loss and our style loss

def mean_squared_error(a, b):
    return tf.reduce_mean(tf.square(a - b))

#it is the average of the sqaure of the difference bwteen the output feature map and our image
# one tensor will be our content image or style image. The other will be our feature map
                #. --> the result of the chosen layer 
# the difeeec is subtracted, the squared then we do a reduce mean
# square makes it a positive

# reduce Mean: find the average of all vlaues inside of the matrix 


In [ ]:
# Creates the loss-function for the content-image.

'''Parameters:
    session: An open TensorFlow session for running the model's graph.
    model: The model, e.g. an instance of the VGG16-class.
    content_image: Numpy float array with the content-image.
    layer_ids: List of integer id's for the layers to use in the model. --> will be ther higher level layers
'''
#you should expirment what looks good for different layers
def create_content_loss(session, model, content_image, layer_ids):

  # it is a python dictionary object that is genrateed with placeholders as keys
  # it genrates a set of place holder values 
  # key is the image, value is the content image.
  # for very time we want to feed anything into our compeiutuon graph -->as ditionary
   feed_dict = model.create_feed_dict(image=content_image)

    # play around with differnet layers 
   layers = model.get_layer_tensors(layer_ids)

  # calaute the output values given the layers --> feed into dictoanry
  # get outptu values for thaose layers for the conent image
   values = session.run(layers, feed_dict=feed_dict)

   with model.graph.as_default():
     # need empty as we will claaues a collection of lossses 
     layer_losses = []


      # For each layer and its corresponding values
     for value, layer in model(values, layers):

        # tf.cocnsstnt --> cannont change
        # this is the content image value
        value_const = tf.constant(value)


        #claaclte loss using mean sqaured error
        # inputs: value at a layer, constant value
        #
        loss = mean_squared_error(layer, value_const)

        # add to pre inititlised list 
        layer_losses.append(loss)


     total_loss = tf.reduce_mean(layer_losses)


   return total_loss


# get eman sqaure eroro of feature activtions of the given layer in th model 
# between the conent image and lmixed image --.~ when minised, it will make the mixed image that much more stylised 



In [ ]:
# for style we do a gram matrix
# gram matrix : we are maeaurifn the correlations between out feture vectors after flattening the filter image into vectors
# it is the matrix product bbetrwn our iniaktl matirix and then its transpoed ( filpping it by 90 degrees )
# i.e multiplying a mtrix by its transpose 
# this is used to minimise via the mean sqaured error
# it is 4D tensoer : collectio of pixel and rgb make it 3d -->a collection of those make it 4D
def gram_matrix(tensor):

  #4d tensor from convolutional layer
  shape = tensor.get_shape()

  # each stack in a the matrox is called a channel.
  num_channels = int(shape[3])


  # -1 : means whatever number makes the data fit
  # we are reshapping th edensor soe it is a 2d matrix 
  # flatten the contents of each o the feature chnnels --> so we can multiplt it 
  # akin to normalisation 
  matrix = tf.reshape(tensor, shape=[-1, num_channels])


    # the 2-dim matrix with itself. 
    # matrix multipled by its transpose
  gram = tf.matmul(tf.transpose(matrix), matrix)

  return gram



In [ ]:
# style loss 
# compute given the gram matrix


def create_style_loss(session, model, style_image, layer_ids):
  # creates place holder value for style image
  feed_dict = model.create_feed_dict(image=style_image)

  # then get layers 
  layers = model.get_layer_tensors(layer_ids)


  with model.graph.as_default():

      # this is deffernece t the contenct loss
      # we take the gram matrix of a layer, instead of the regular
      # why ? ( from crator at a confernce ): it encodes 2nd order ststisics of the set of filters --> looking at it froma higehr layer
      # gram maxtrix , tolss away anything uncewesay to focus on the syle 
      # hence it works for style and not content 
    gram_layers = [gram_matrix(layer) for layer in layers]


    values = session.run(gram_layers, feed_dict=feed_dict)


    # Initialize an empty list of loss-functions.
    layer_losses = []


    # For each Gram-matrix at each layer and its corresponding values or their loss function.
    for value, gram_layer in model(values, gram_layers):

      value_const = tf.constant(value)


      # where calducalting the mean sqaured error of the ground layer and the value of that ground matrix layer when inputting the style image
      loss = mean_squared_error(gram_layer, value_const)


      # list of loss-functions --> losses at each layer
      layer_losses.append(loss)
    
    # take the average of all losess in matrix --> scalar value 
    total_loss = tf.reduce_mean(layer_losses)

  return total_loss





In [ ]:
# one more loss 
# the denoising loss( totoal vartioan loss) --> when paper didnt do this image was OK
                                          # when they did it was --> result improved
# shitfs images by one pixel on the x and y axis and calculates the diffenec btween the shifted image the original image 
# abosloute value so it is positive 

#--> helps supress noise in the image that was generated : its a blurriness, not that clear 


def create_denoise_loss(model):
    loss = tf.reduce_sum(tf.abs(model.input[:,1:,:,:] - model.input[:,:-1,:,:])) + \
           tf.reduce_sum(tf.abs(model.input[:,:,1:,:] - model.input[:,:,:-1,:]))

    return loss

# Style-transfer 

In [ ]:
# we can modify the weigths 
# weighing the sytle more than the content
# To:DO --> learn the best weights later with learnig algorthim 
'''
parameters: 
    content_image: Numpy 3-dim float-array with the content-image.
    style_image: Numpy 3-dim float-array with the style-image.
    content_layer_ids: List of integers identifying the content-layers.
    style_layer_ids: List of integers identifying the style-layers.
    weight_content: Weight for the content-loss-function.
    weight_style: Weight for the style-loss-function.
    weight_denoise: Weight for the denoising-loss-function.
    num_iterations: Number of optimization iterations to perform.
    step_size: Step-size for the gradient in each iteration.

'''
def style_transfer(content_image, style_image,
                   content_layer_ids, style_layer_ids,
                   weight_content=1.5, weight_style=10.0,
                   weight_denoise=0.3,
                   num_iterations=120, step_size=10.0):
  

  #model = vgg16.VGG16()

  model = VGG16()

  # Create a TensorFlow-session.
  session = tf.InteractiveSession(graph=model.graph)

  # Print the names of the content-layers.
  print("Content layers:")
  print(model.get_layer_names(content_layer_ids))
  print()


  # Create the loss-function for the content-layers and -image.
  loss_content = create_content_loss(session=session,
                                       model=model,
                                       content_image=content_image,
                                       layer_ids=content_layer_ids)
  
   # Create the loss-function for the denoising of the mixed-image.
  loss_denoise = create_denoise_loss(model)


  




